# Classification of the MNIST-dataset

Classification of the images in the hand-drawn MNIST-dataset has been a benchmark for models for years. The dataset contains about 60 000 training images and 10 000 test images. This might seem like a lot but considering the fact that the average neural network used for classification on this dataset is comprised of about 200 000 parameters, it is rather interesting to get the accuracy we do. In this notebook, I will be exploring the accuracy of predictions of a basic neural network and one with convolutions.

This notebook is the first Convolutional Neural Network I have written and as such, the daunting task of tuning the layers proved quite a struggle. The following, however, was the process I went through:

#### Convolutions:

Almost all CNNs start with a convolution layer. We aim to extract special patterns (or features) from the images to better emulate the way humans classify images. These convolutions come with quite a lot of tuning, however. We start with channels. The MNIST-dataset is grayscale and thus each image has one channel. The out-channel is the amount of filters to be applied through the layer and can be conceptualized as the amount of different patterns we would like to learn in this layer. Increasing this hyperparameter increases the complexity of the model and can lead to overfitting. It is common to choose 16 or 32 but without some experimenting, the best choice is unclear. Next, we have to choose the size of the filter, how much it moves and padding of the image. Using reference (2), we can determine the hyperparameters such that all pixels on the image are used to the same magnitude. The size will most likely be 3x3 with our image size (28x28), then with stride of 1, we need padding of 1 on each side.

#### Batch-Normalization and Dropout
A quick search on google scholar on the importance of BN and Dropout and where to put them highlights the ongoing debate on this matter. Dropout is the process of choosing neurons at random and ignoring their output. This process aims to reduce overfitting by encouraging the network to rely on multiple neurons for specific patterns. Batch-Normalization aims to increase the convergence rate on a minimal loss by decreasing the internal covariance shift. The original paper written by Loffe and Szegedy suggests putting Batch normalization before activation functions. This, however doesn't make sense as it leads defies the point of BN, as supported by reference 4. Also, reference 5 (and many tests) suggest that using dropout regularization with BN leads to worse results compared to using only BN and as such, BN will be used without DR and the BN-layers will be put after the activation layers.


In [1]:
import MNIST_classifier as mn
from torch import nn
import torch
from torch.utils.data import TensorDataset, DataLoader
training_dataloader, testing_dataloader = mn.init_MNIST(True)


2024-05-04 23:25:26.697909: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 23:25:27.545250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


0.13066047430038452
0.30810782313346863


In [2]:
model = mn.CNN().cuda()
#model.train_from_dataset(1, training_dataloader, testing_dataloader)

In [3]:
from sklearn.datasets import fetch_openml
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
mnist = fetch_openml('mnist_784', version=1)

# Get the data and target
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
X_train = torch.tensor(X_train.astype(float).values)
X_test = torch.tensor(X_test.astype(float).values)
y_train = torch.tensor(y_train.astype(int).values)
y_test = torch.tensor(y_test.astype(int).values)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=512, shuffle=True)

datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1
        )
model.train_from_generator(40000, datagen, X_train, y_train, test_dataloader)

Test Error: 
 Accuracy: 99.2%, Avg loss: 0.028180 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.025369 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.025463 

Test Error: 
 Accuracy: 99.2%, Avg loss: 0.027369 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.023209 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.021743 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.023018 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.024319 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.021547 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.024599 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.026931 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.024250 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.022894 

Test Error: 
 Accuracy: 99.5%, Avg loss: 0.021348 

Test Error: 
 Accuracy: 99.3%, Avg loss: 0.020978 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.021048 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.022512 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.025932 

Test Error: 
 Accuracy: 99.4%, Avg loss: 0.022672 

Test Error: 

In [4]:
# class CNN(nn.Module):
    
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.device = "cuda" if torch.cuda.is_available else "cpu"
#         self.features = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3), stride=1, padding=1),
#             nn.ReLU(),
#             nn.BatchNorm2d(16),
#             nn.MaxPool2d(2),
#             nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3),stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.BatchNorm2d(32),
#             nn.MaxPool2d(2)
            
#         )
#         self.classify = nn.Sequential(
#             nn.Linear(in_features=7*7*32, out_features=256),
#             nn.ReLU(),
#             nn.BatchNorm1d(256),
#             nn.Linear(in_features=256, out_features=10)         
#         )
        

    # def forward(self, x):
    #     x = self.features(x)
    #     x = x.view(-1, 32*7*7)
    #     x = self.classify(x)
    #     return x

In [5]:
torch.save(model.state_dict(), 'final_model.pth')

## References

<ol>
<li>Zhang, A., Lipton, Z., li, M. and Smola, A. (2023). Dive Into Deep Learning. [online] Cambridge University Press. Available at: https://d2l.ai.</li>
<li>asiltureli.github.io. (n.d.). ConvNet Size Calculator. [online] Available at: https://asiltureli.github.io/Convolution-Layer-Calculator/ [Accessed 4 May 2024].</li>
<li>Loffe, S. and Szegedy, C. (2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. [online] Available at: https://arxiv.org/abs/1502.03167 [Accessed 4 May 2024].</li>
<li>Rosebrock, A. (2021). Convolutional Neural Networks (CNNs) and Layer Types. [online] PyImageSearch. Available at: https://pyimagesearch.com/2021/05/14/convolutional-neural-networks-cnns-and-layer-types/.</li>
<li>Li, X., Chen, S., Hu, X. and Yang, J. (2018). Understanding the Disharmony between Dropout and Batch Normalization by Variance Shift. [online] Available at: https://arxiv.org/pdf/1801.05134 [Accessed 4 May 2024].</li>
</ol>